In [ ]:
%matplotlib inline
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl

from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [ ]:
fig_path = pl.Path("../figures")
fig_path.mkdir(exist_ok=True, parents=True)
ani_path = pl.Path("../animation")
ani_path.mkdir(exist_ok=True, parents=True)

In [ ]:
def custom_print(text):
    """Prints the provided text without advancing."""
    print(256*" ", end="\r")
    print(text, end="\r")

# Constants

In [ ]:
cps = 1100.0  # Zone 1 heat capacity ($\frac{J}{kg \cdot ^{\circ} C}$)
cpw = 4180.0  # Heat capacity of water ($\frac{J}{kg \cdot ^{\circ} C}$)
rhow = 1000.0  # Density of water ($kg/m^3$)
rhos = 2500.0  # Density of dry solid aquifer material ($kg/m^3$)
lhv = 2500.0  # Latent heat of vaporization ($kJ/kg$)
gravity = 3.7  # Gravity ($m/s^2$)
viscosity = 0.001  # Viscosity ($Pa s$)
alpha = 5e-10  # rock matrix compressibility ($1/Pa$)
beta = 4.8e-10  # fluid compressibility ($1/Pa$)

adv_scheme = "UPSTREAM"

al = 0.1  # Longitudinal dispersivity ($m$)
ath1 = 0.01  # Transverse dispersivity ($m$)
kts = 3.0  # Zone 1 thermal conductivity ($\frac{W}{m \cdot ^{\circ} C}$)
ktw = 0.58  # Thermal conductivity of water ($\frac{W}{m \cdot ^{\circ} C}$)

Tsurf = 0.0
Trech = 5.0

denseslp = -0.375 # Density and temperature slope

# Read the elevation data

In [ ]:
fpath = pl.Path("../data/model_elevations.csv")

In [ ]:
elevation_data = np.genfromtxt(fpath, names=True, delimiter=",")

In [ ]:
elevation_data.dtype.names

# Define the problem dimensions

In [ ]:
km2m = 1000.0
len_x = 10000.0 * km2m
len_z = 20.0 * km2m

In [ ]:
nlay = 20
nrow = 1
ncol = elevation_data["x"].shape[0]
nlay, nrow, ncol

In [ ]:
delx, dely = len_x / ncol, 1.0
delx, dely

In [ ]:
sec2day = 86400.0
years2days = 365.25
sim_length_years = 2e8
sim_length = sim_length_years * years2days
step_length_years = sim_length_years / 100
dtmax = step_length_years * years2days
nstp = sim_length_years / step_length_years
nper = 1
tdis_data = [(sim_length, nstp, 1.0)]
dtmax, tdis_data

# Solver settings

In [ ]:
outer_maximum=500
inner_maximum=200
inner_dvclose=1e-6 
outer_dvclose=1e-4

# Scale the cell center and elevation data using problem dimensions

In [ ]:
x = elevation_data["x"] * len_x

In [ ]:
datum = 0.6 * len_z

In [ ]:
h0 = 0.415 * len_z - datum

In [ ]:
top = elevation_data["top"] * len_z - datum

In [ ]:
bot = elevation_data["bottom"] * len_z - datum

In [ ]:
cryo = elevation_data["cryosphere"] * len_z - datum

In [ ]:
rech = elevation_data["recharge"] * len_z - datum

# Calculate the bottom of each layer and the elevation of each node

In [ ]:
thickness = top - bot
dz = thickness / nlay

In [ ]:
botm = [top - dz * (k + 1) for k in range(nlay)]

In [ ]:
znode = [top - b + dz / 2 for b in botm]

# Plot the model domain

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        layout="constrained",
        figsize=(8, 3),
        )
    ax.set_xlim(0, len_x)
    ax.set_ylim(-13000, 9000)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.fill_between(x, top, y2=cryo, color="green", label="Cryosphere")
    ax.fill_between(x, bot, y2=h0, color="cyan", label="Initial head")
    for b in botm[:-1]:
        ax.plot(x, b, color="black", lw=0.75, label=None)
    ax.axhline(0, color="black", ls="--", label=None)
    ax.plot(-100, 0, color="black", lw=0.75, label="Model layers")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.set_xlabel("x-coordinate, m")
    ax.set_ylabel("Relative elevation, m")
    flopy.plot.styles.graph_legend(ax=ax, ncol=2, title="", loc="lower left", labelspacing=0.15)
    
    fig.savefig(fig_path / f"conceptual_model.png", dpi=300, transparent=True)
    

# Calculate the depth-dependent permeability, porosity, and specific storage

In [ ]:
10**(-12.65 - 3.2 * np.log10(1))

In [ ]:
logk = np.array([-12.65 - 3.2 * np.log10(z / km2m) for z in znode]).reshape(nlay, nrow, ncol)

In [ ]:
10**logk.min(), 10**logk.mean(), 10**logk.max()

In [ ]:
K = 86400.0 * rhow * gravity * 10**logk / 1e-3 # m/d

In [ ]:
logn = np.array([-1.65 - 0.8 * np.log10(z / km2m) for z in znode]).reshape(nlay, nrow, ncol)

In [ ]:
10**logn.mean()

In [ ]:
porosity = 10**logn

In [ ]:
ss = rhow * gravity * (alpha + porosity * beta)
ss.min(), ss.mean(), ss.max()

# Create the base model

In [ ]:
ws = pl.Path("../run/")
name = "mars"

In [ ]:
gwf_name = f"{name}_gwf"
gwe_name = f"{name}_gwe"

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws)

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_data)

In [ ]:
dtmin = 1. * years2days
dt = dtmax

In [ ]:
# atsperiod = [
#     (iper, dtmin, dtmax, dt, 2.0, 5.0) for iper in range(nper)
# ]
# tdis.ats.initialize(
#     maxats=len(atsperiod),
#     perioddata=atsperiod,
#     filename=f"{name}.ats",
# )

In [ ]:
imsgwf = flopy.mf6.ModflowIms(
    sim, 
    complexity="simple", 
    linear_acceleration="bicgstab", 
    inner_dvclose=inner_dvclose, 
    outer_dvclose=outer_dvclose,
    outer_maximum=outer_maximum,
    inner_maximum=inner_maximum,
    print_option="SUMMARY",
    filename=f"{gwf_name}.ims"
)

## GWF model

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwf_name, newtonoptions="under_relaxation")

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, delr=delx, delc=dely, nlay=nlay, nrow=nrow, ncol=ncol, top=top, botm=botm, filename=f"{gwf_name}.dis")

### Plot hydraulic properties using the gwf model

In [ ]:
extent = (0, len_x, -13000, 9000)

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(
        ncols=1,
        nrows=2,
        layout="constrained",
        figsize=(8, 6),
        sharex=True,
        )
    ax = axs[0]
    xs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pk = xs.plot_array(logk)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)
    ax.set_ylabel("Relative elevation, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pk, cax=cax, orientation='horizontal')
    cbar.set_label(r"$\text{log} k\text{, m}^2$")
    flopy.plot.styles.add_text(ax=ax, x=0.01, y=0.05, ha="left", va="bottom", text=r"$\text{log} k = -12.65 - 3.2 \text{log} z$", bold=False, italic=False)
    flopy.plot.styles.graph_legend(ax=ax, ncol=1, title="", loc="upper right", labelspacing=0.15)
    
    ax = axs[1]
    xs1 = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pn = xs1.plot_array(logn)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)
    ax.set_ylabel("Relative elevation, m")
    ax.set_xlabel("x-coordinate, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pn, cax=cax, orientation='horizontal')
    cbar.set_label(r"$\text{log} \theta\text{, unitless}$")
    flopy.plot.styles.add_text(ax=ax, x=0.01, y=0.05, ha="left", va="bottom", text=r"$\text{log} \theta = -1.65 - 0.8 \text{log} z$", bold=False, italic=False)

    fig.savefig(fig_path / f"model_hydraulic.png", dpi=300, transparent=True)
    

### Build the rest of the flow model

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, k=K, icelltype=1, filename=f"{gwf_name}.npf")

In [ ]:
sto = flopy.mf6.ModflowGwfsto(gwf, sy=porosity, ss=ss, transient={0: True}, iconvert=1, filename=f"{gwf_name}.sto")

In [ ]:
# buy = flopy.mf6.ModflowGwfbuy(gwf, denseref=rhow, packagedata=[(0, denseslp, 0.0, gwe_name, "temperature")])

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=h0, filename=f"{gwf_name}.ic")

In [ ]:
rch_rate = 2e-10 * years2days
rch_spd = []
for idx, r in enumerate(rech):
    if not np.isnan(r):
        rch_spd.append((0, 0, idx, rch_rate, Trech))

In [ ]:
rch = flopy.mf6.ModflowGwfrch(gwf, auxiliary="TEMPERATURE", stress_period_data=rch_spd, pname="RCH", filename=f"{gwf_name}.rch")

In [ ]:
drn_spd = []
for idx, t in enumerate(top):
    if t < 0.0:
        cond = K[0, 0, idx] * delx * dely / znode[0][idx]
        drn_spd.append((0, 0, idx, float(t), float(cond)))
        

In [ ]:
drn = flopy.mf6.ModflowGwfdrn(gwf, stress_period_data=drn_spd, pname="DRN", filename=f"{gwf_name}.drn")

In [ ]:
ocgwf = flopy.mf6.ModflowGwfoc(
    gwf, 
    head_filerecord=f"{gwf_name}.hds", 
    saverecord=[("HEAD", "FREQUENCY", "1"), ("HEAD", "LAST")], 
    printrecord=[("BUDGET", "FREQUENCY", "10")], 
    filename=f"{gwf_name}.oc"
)

## GWE model

In [ ]:
gwe = flopy.mf6.ModflowGwe(sim, modelname=gwe_name)

In [ ]:
disgwe = flopy.mf6.ModflowGwedis(gwe, delr=delx, delc=dely, nlay=nlay, nrow=nrow, ncol=ncol, top=top, botm=botm, filename=f"{gwe_name}.dis")

In [ ]:
T0 = np.zeros((nlay, nrow, ncol), dtype=float)
for k, arr in enumerate(botm):
    for idx, zb in enumerate(arr):
        z = zb + dz[idx] / 2.0
        dwt = cryo[idx] - h0
        dtdwt = 5.0 / dwt
        if z > cryo[idx]:
            temp = 0.0
        elif z < h0:
            temp = 5.0
        else:
            temp = (cryo[idx] - z) * dtdwt
        T0[k, 0, idx] = temp

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        layout="constrained",
        figsize=(8, 3),
        sharex=True,
        )
    xs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pk = xs.plot_array(T0)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)
    ax.set_ylabel("Relative elevation, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pk, cax=cax, orientation='horizontal')
    cbar.set_label(r"Initial relative temperature ${\circ}K$")
    flopy.plot.styles.graph_legend(ax=ax, ncol=1, title="", loc="upper right", labelspacing=0.15)

    fig.savefig(fig_path / f"model_initial_temperature.png", dpi=300, transparent=True)

In [ ]:
icgwe = flopy.mf6.ModflowGweic(gwe, strt=T0, pname="IC", filename=f"{gwe_name}.ic")

In [ ]:
advgwe = flopy.mf6.ModflowGweadv(gwe, scheme=adv_scheme, pname="ADV", filename=f"{gwe_name}.adv")

In [ ]:
gweest = flopy.mf6.ModflowGweest(
    gwe,
    porosity=porosity,
    heat_capacity_water=cpw,
    density_water=rhow,
    latent_heat_vaporization=lhv,
    heat_capacity_solid=cps,
    density_solid=rhos,
    pname="EST",
    filename=f"{gwe_name}.est",
    )

In [ ]:
cnd = flopy.mf6.ModflowGwecnd(
    gwe,
    xt3d_off=True,
    alh=al,
    ath1=ath1,
    ktw=ktw,
    kts=kts,
    pname="CND",
    filename=f"{gwe_name}.cnd",
    )

In [ ]:
sourcerecarray = [
    ("RCH", "AUX", "TEMPERATURE"),
    ]
ssm = flopy.mf6.ModflowGwessm(
    gwe, sources=sourcerecarray, pname="SSM", filename=f"{gwe_name}.ssm"
    )

In [ ]:
geothermal_gradient = 0.030 # geothermal gradient ($W/m^2$)
esrc = [(nlay - 1, 0, idx, geothermal_gradient * sec2day) for idx in range(ncol)]

In [ ]:
esl = flopy.mf6.ModflowGweesl(
    gwe,
    maxbound=len(esrc),
    stress_period_data=esrc,
    pname="ESL",
    filename=f"{gwe_name}.esl",
    )

In [ ]:
ctp = []
for idx, r in enumerate(rech):
    if np.isnan(r):
        ctp.append((0, 0, idx, Tsurf))
    else:
        ctp.append((0, 0, idx, Trech))
        

In [ ]:
ctp = flopy.mf6.ModflowGwectp(
    gwe,
    maxbound=len(ctp),
    stress_period_data=ctp,
    pname="CTP",
    filename=f"{gwe_name}.ctp",
)

In [ ]:
ocgwe = flopy.mf6.ModflowGweoc(
    gwe,
    temperature_filerecord=f"{gwe_name}.bin", 
    saverecord=[("TEMPERATURE", "FREQUENCY", "1"), ("TEMPERATURE", "LAST")], 
    printrecord=[("BUDGET", "FREQUENCY", "10")], 
    filename=f"{gwe_name}.oc"
)

In [ ]:
imsgwe = flopy.mf6.ModflowIms(
    sim, 
    complexity="simple", 
    linear_acceleration="bicgstab", 
    inner_dvclose=inner_dvclose, 
    outer_dvclose=outer_dvclose,
    outer_maximum=outer_maximum,
    inner_maximum=inner_maximum,
    print_option="SUMMARY",
    filename=f"{gwe_name}.ims"
)

In [ ]:
sim.register_ims_package(imsgwe, [gwe_name])

In [ ]:
    # GWF GWE exchange
gwfgwe = flopy.mf6.ModflowGwfgwe(
    sim,
    exgtype="GWF6-GWE6",
    exgmnamea=gwf_name,
    exgmnameb=gwe_name,
    filename=f"{name}.gwfgwe",
    )

# Write model datasets and run the model

In [ ]:
sim.write_simulation()

In [ ]:
success, buff = sim.run_simulation(silent=False, custom_print=custom_print)
print(f"\n{success}")

# Animate results

In [ ]:
ani_ext = ".mp4"
Writer = mpl.animation.writers["ffmpeg"]
writer = Writer(fps=4, metadata=dict(artist="jdhughes"), bitrate=2056)

In [ ]:
output_times = gwf.output.head().get_times()
ntimes = len(output_times)
frames = np.arange(1, ntimes, dtype=int)
ntimes

In [ ]:
hmin, hmax = 1e20, -1e20
for totim in output_times:
    head = gwf.output.head().get_data(totim=totim)
    hmin = min(hmin, head[head <= 1e20].min())
    hmax = max(hmax, head[head <= 1e20].max())    
hmin, hmax

In [ ]:
tmin, tmax = 1e20, -1e20
for totim in output_times:
    temp = gwe.output.temperature().get_data(totim=totim)
    tmin = min(tmin, temp[temp <= 1e20].min())
    tmax = max(tmax, temp[temp <= 1e20].max())    
tmin, tmax

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        layout="constrained",
        figsize=(8, 3),
        )
    
    totim = output_times[0]
    toyears = int(totim / years2days)
    title_str = f"{toyears:>,} years"
    
    head = gwf.output.head().get_data(totim=totim)
    
    xs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pk = xs.plot_array(head, head=head, vmin=hmin, vmax=hmax)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)

    sim_text = flopy.plot.styles.add_text(ax=ax, text=title_str, x=0.995, y=0.03, ha="right", va="bottom")
    
    ax.set_ylabel("Relative elevation, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pk, cax=cax, orientation='horizontal')
    cbar.set_label("Head, m")
    flopy.plot.styles.graph_legend(ax=ax, ncol=1, title="", loc="upper right", labelspacing=0.15)

    def func(idx):
        global pk
        totim = output_times[idx]
        toyears = int(totim / years2days)
        title_str = f"{toyears:>,} years"

        sim_text.set_text(title_str)
        
        head = gwf.output.head().get_data(totim=totim)

        pk.remove()
        pk = xs.plot_array(head, head=head, vmin=hmin, vmax=hmax)

        return pk

    ani = FuncAnimation(fig, func, frames=frames, blit=False)
    #HTML(ani.to_jshtml())    
    ani.save(ani_path / f"head_results_gwf{ani_ext}", writer=writer)

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        layout="constrained",
        figsize=(8, 3),
        )
    
    totim = output_times[0]
    toyears = int(totim / years2days)
    title_str = f"{toyears:>,} years"
    
    temp = gwf.output.temperature().get_data(totim=totim)
    
    xs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0}, extent=extent)
    pk = xs.plot_array(temp, vmin=tmin, vmax=tmax)
    ax.plot(x, top, color="red", lw=3., label="Specified temperature")
    ax.plot(x, rech, color="blue", lw=3., label="Recharge")
    ax.plot(x, bot, color="orange", lw=3, label="Geothermal gradient")
    ax.axhline(0, color="black", ls="--", label=None)

    sim_text = flopy.plot.styles.add_text(ax=ax, text=title_str, x=0.995, y=0.03, ha="right", va="bottom")
    
    ax.set_ylabel("Relative elevation, m")
    cax = ax.inset_axes([0.15, 0.2, 0.3, 0.04])
    cbar = fig.colorbar(pk, cax=cax, orientation='horizontal')
    cbar.set_label(r"Relative temperature, $^{\circ}K$")
    flopy.plot.styles.graph_legend(ax=ax, ncol=1, title="", loc="upper right", labelspacing=0.15)

    def func(idx):
        global pk, sim_text
        totim = output_times[idx]
        toyears = int(totim / years2days)
        title_str = f"{toyears:>,} years"

        sim_text.set_text(title_str)
        
        temp = gwf.output.temperature().get_data(totim=totim)

        pk.remove()
        pk = xs.plot_array(temp, vmin=tmin, vmax=tmax)

        return pk, sim_text

    ani = FuncAnimation(fig, func, frames=frames, blit=False)
    #HTML(ani.to_jshtml())    
    ani.save(ani_path / f"temperature_results_gwe{ani_ext}", writer=writer)